In [1]:
import pandas as pd
from google_trans_new import google_translator
translator = google_translator()  

In [46]:
socioecon = pd.read_csv('../data/Socio-economic Data/socioecon_clean.csv')

### Re-arrange some indicators

In [47]:
socioecon.keys()

Index(['file_name', 'table_name', 'indicator', 'category_1', 'category_2',
       'category_3', 'Region', 'region_type', 'Year', 'value', 'units'],
      dtype='object')

In [48]:
socioecon.replace({'indicator': {'Reservoirs': 'Number of Reservoirs'}, 'category_1': {'Number of Reservoirs  (unit)': 'Total'}}, inplace=True)
socioecon.loc[(socioecon['indicator']=='Capacity of Reservoirs')&(socioecon['category_1']=='Total'), 'units'] = '100m m3'
socioecon.loc[(socioecon['indicator']=='Capacity of Reservoirs')&(socioecon['category_1']=='Total'), 'units'] = '100m m3'

In [49]:
socioecon.drop(socioecon.loc[(socioecon['indicator']=='Employment')&(socioecon['category_1']=='Terciary Industry')].index, inplace=True)
socioecon.loc[(socioecon['indicator']=='Employed persons')&(socioecon['category_1']=='Industry breakdown'), 'indicator'] = 'Employed persons (Industry breakdown)'
socioecon.loc[(socioecon['indicator']=='Employed persons')&(socioecon['category_1']=='Urban/Rural breakdown'), 'indicator'] = 'Employed persons (Urban/Rural breakdown)'
socioecon.loc[(socioecon['file_name']=='Employment_就业')&(socioecon['indicator']=='Employed People'), 'indicator'] = 'Employed People in urban areas'
for item in ['Employed persons (Industry breakdown)', 'Employed persons (Urban/Rural breakdown)', 'Employed People in urban areas']:
    socioecon.loc[socioecon['indicator']==item, 'category_1'] = socioecon.loc[socioecon['indicator']==item]['category_2']
    socioecon.loc[socioecon['indicator']==item, 'category_2'] = socioecon.loc[socioecon['indicator']==item]['category_3']
    socioecon.loc[socioecon['indicator']==item, 'category_3'] = None

In [50]:
summary_socio = socioecon.groupby(['file_name', 'indicator', 'category_1', 'category_2',
       'category_3'], sort=False, as_index=False, dropna=False).size()

In [51]:
for indicator in ['GDP', 'GDP (constant price)', 'GDP (compared to 1978)', 'GDP (main contributors to increase)', 'GDP (main contributors to growth)', 'GRP',
                 'GRP (compared to preceding year)']:
    nindicator = indicator.replace('GDP', 'Gross Domestic Product').replace('GRP', 'Gross Regional Product')
    for item in ['Industry breakdown', 'Sector']:
            socioecon.loc[(socioecon['file_name']=='GDP_国内生产总值')&(socioecon['indicator']==indicator)& (socioecon['category_1']==item), 'indicator'] = f'{nindicator} ({item})'
            socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})', 'category_1'] = socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})']['category_2']
            socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})', 'category_2'] = socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})']['category_3']
            socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})', 'category_3'] = None
socioecon.drop(socioecon.loc[(socioecon['file_name']=='GDP_国内生产总值')&(socioecon['indicator']==indicator)].index, inplace=True)

In [52]:
for item in ['By Gender', 'By Residence']:
    socioecon.loc[(socioecon['file_name']=='Population_人口')&(socioecon['indicator']=='Population')& (socioecon['category_1']==item), 'indicator'] = f'Population ({item})'
    socioecon.loc[socioecon['indicator']==f'Population ({item})', 'category_1'] = socioecon.loc[socioecon['indicator']==f'Population ({item})']['category_2']
    socioecon.loc[socioecon['indicator']==f'Population ({item})', 'category_2'] = socioecon.loc[socioecon['indicator']==f'Population ({item})']['category_3']
    socioecon.loc[socioecon['indicator']==f'Population ({item})', 'category_3'] = None

socioecon.loc[socioecon['indicator']=='Dependency Rate', 'category_1'] = socioecon.loc[socioecon['indicator']=='Dependency Rate']['category_2']
socioecon.loc[socioecon['indicator']=='Dependency Rate', 'category_2'] = socioecon.loc[socioecon['indicator']=='Dependency Rate']['category_3']
socioecon.loc[socioecon['indicator']=='Dependency Rate', 'category_3'] = None

In [53]:
ie_inds = summary_socio[summary_socio['file_name']==summary_socio['file_name'].unique()[9]]['indicator'].unique()
ie_inds

array(['Length of Railways in Operation', 'Length of Highways',
       'Length of Navigable Inland Waterways',
       'Length of Regular Civil Aviation Routes',
       'Length of Petroleum and Gas Pipelines', 'Passenger Traffic',
       'Passenger Traffic per kilometer', 'Freight Traffic',
       'Travel Distance per passenger',
       'Average Freight Transport Distance',
       'Highspeed Railway in Operation', 'Highspeed Passenger Traffic',
       'Civil Vehicles', 'Private Vehicles', 'Air Passenger Traffic',
       'Air Freight Traffic', 'Total Air Traffic',
       'Flying Time of General Aviation'], dtype=object)

In [54]:
socioecon.loc[(socioecon['indicator']=='Civil Vehicles')&((socioecon['category_1']=='Motor Drivers')), 'indicator'] = 'Motor Drivers'
socioecon.loc[socioecon['indicator']=='Motor Drivers', 'category_1'] = socioecon.loc[socioecon['indicator']=='Motor Drivers']['category_2']
socioecon.loc[socioecon['indicator']=='Motor Drivers', 'category_2'] = None
socioecon.loc[socioecon['indicator']=='Motor Drivers', 'category_3'] = None

### Fix namings and regions

In [55]:
cat_dict = {'Number of Irrigated Areas over 10 000 Mu (set)': 'Number of Irrigated Areas over 10 000 Mu',
           'Effective Irrigated Area (10 000 hectares)': '10 000 hectares',
           'Number of Reservoirs (unit)': 'Total',
           'Capacity of Reservoirs (100 million cu.m)': 'Total',
           'Final Consumption Rate (%)': 'Final Consumption Rate',
           'Capital Formation Rate (%)': 'Capital Formation Rate',
           'Commodities and Transactions not classified elsewhere   in the SITC': 'Commodities and Transactions not classified elsewhere in the SITC',
           'Aquatic and Seawater Products(10 000 tons)': 'Aquatic and Seawater Products',
           'Synthetic Fibers Suitable for Spinning(10 000 tons)': 'Synthetic Fibers Suitable for Spinning',
           'Manganese Ores and Concentrates(10 000 tons)': 'Manganese Ores and Concentrates',
           'Chromium Ores and Concentrates(10 000 tons)': 'Chromium Ores and Concentrates',
           'Xerox and Hectograph Printing Equipment(10 000 sets)': 'Xerox and Hectograph Printing Equipment'}
socioecon.replace({'category_1': cat_dict}, inplace=True)

In [56]:
socioecon.drop(socioecon.loc[(socioecon['Region']=='a) Since 1980, the difference between the Gross Domestic Product and the Gross National Income (formerly, the Gross National Product) is the ')].index, inplace=True)
socioecon.drop(socioecon.loc[(socioecon['Region']=='    net income of primary distribution from the rest of the world.')].index, inplace=True)

In [57]:
#fix spaces in region names
reg_dict = {}
for item in list(socioecon['Region'].unique()):
    try:
        new_label = re.sub("\s\s+", " ", item)
        if new_label[0] == ' ':
            new_label = new_label[1:]
        if new_label[-1] == ' ':
            new_label = new_label[:-1]
        if item != new_label:
            reg_dict[item] = new_label
    except:
        pass

socioecon.replace({'Region':reg_dict}, inplace=True)

In [58]:
#drop rows where value = 'nan'
print(len(socioecon))
socioecon.dropna(subset=['value'], inplace=True)
print(len(socioecon))

16403
16123


### rename columns and include string translations

In [59]:
ren_dict = {'file_name': 'subgroup_en', 'Year': 'year'}
for item in socioecon.keys()[2:]:
    if item not in ['Year', 'region_type', 'value']:
        ren_dict[item] = f'{item.lower()}_en'
socioecon.rename(columns=ren_dict, inplace=True)
socioecon.drop(columns=['table_name'], inplace=True)

In [ ]:
socioecon['indicator_en'].replace({'GDP': 'Gross Domestic Product',
                                   'GNI': 'Gross National Income'}, regex=True, inplace=True)

In [60]:
socioecon['subgroup_cn'] = [x.split('_')[1] for x in socioecon['subgroup_en']]
socioecon['subgroup_en'] = [x.split('_')[0] for x in socioecon['subgroup_en']]
socioecon['group_en'] = 'Socioeconomic'
socioecon['group_cn'] = '社会经济'

In [117]:
inds = ''
for x in socioecon['indicator_en'].unique():
    inds += f'. {x}'
translated = translator.translate(inds,lang_tgt='zh-cn') 

In [118]:
ind_dict = {}
for en, cn in zip(inds.split('. ')[1:], translated.split('。')[1:]):
    ind_dict[en] = cn

In [121]:
socioecon['indicator_cn'] = socioecon['indicator_en']
socioecon['indicator_cn'].replace(ind_dict, inplace=True)

In [126]:
for field in range(1,4):
    labels = ''
    for x in socioecon[f'category_{field}_en'].unique():
        labels += f'. {x}'
    translated = translator.translate(labels,lang_tgt='zh-cn')
    lab_dict = {}
    for en, cn in zip(labels.split('. ')[1:], translated.split('。')[1:]):
        lab_dict[en] = cn
    socioecon[f'category_{field}_cn'] = socioecon[f'category_{field}_en']
    socioecon[f'category_{field}_cn'].replace(ind_dict, inplace=True)

In [129]:
for field in ['units', 'region']:
    labels = ''
    for x in socioecon[f'{field}_en'].unique():
        labels += f'. {x}'
    translated = translator.translate(labels,lang_tgt='zh-cn')
    lab_dict = {}
    for en, cn in zip(labels.split('. ')[1:], translated.split('。')[1:]):
        lab_dict[en] = cn
    socioecon[f'{field}_cn'] = socioecon[f'{field}_en']
    socioecon[f'{field}_cn'].replace(ind_dict, inplace=True)

In [132]:
socioecon = socioecon[['group_en', 'group_cn', 'subgroup_en', 'subgroup_cn',
                      'indicator_en', 'indicator_cn',
                       'category_1_en', 'category_1_cn',
                       'category_2_en', 'category_2_cn',
                       'category_3_en','category_3_cn',
                       'region_en', 'region_cn', 'region_type',
                      'year', 'value', 'units_en', 'units_cn']]

### fixing totals

In [189]:
total_inds = []
for indicator in socioecon['indicator'].unique():
    if len(socioecon[socioecon['indicator']==indicator]['category_1'].unique()) > 1 and 'Total' in socioecon[socioecon['indicator']==indicator]['category_1'].unique():
        total_inds.append(indicator)

In [190]:
df = socioecon[socioecon['indicator']==total_inds[0]]
a = df.groupby('category_1').sum()

In [193]:
total_inds

['Consumption of Chemical Fertilizers',
 'Number of Reservoirs',
 'Capacity of Reservoirs',
 'Building Floorspace',
 'Employment',
 'Employed People in urban areas',
 'International Trade of Goods',
 'Exports by SITC',
 'Imports by SITC',
 'International Trade (by region)',
 'Length of Railways in Operation',
 'Length of Highways',
 'Length of Regular Civil Aviation Routes',
 'Passenger Traffic',
 'Passenger Traffic per kilometer',
 'Freight Traffic',
 'Travel Distance per passenger',
 'Average Freight Transport Distance',
 'Civil Vehicles',
 'Motor Drivers',
 'Private Vehicles',
 'Air Passenger Traffic',
 'Air Freight Traffic',
 'Total Air Traffic',
 'Flying Time of General Aviation']

In [199]:
socioecon[socioecon['indicator']=='Employment']['category_1'].unique()

array(['Total', 'Primary Industry', 'Secondary Industry',
       'Tertiary Industry', 'Urban', 'Rural'], dtype=object)

In [264]:
socioecon[socioecon['indicator']=='Employment']

,file_name,table_name,indicator,category_1,category_2,category_3,Region,region_type,Year,value,units
934,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1952.0,20729.000000,10 000 persons
935,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1957.0,23771.000000,10 000 persons
936,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1962.0,25910.000000,10 000 persons
937,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1965.0,28670.000000,10 000 persons
938,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1970.0,34432.000000,10 000 persons
...,...,...,...,...,...,...,...,...,...,...,...
1821,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2014.0,49.100000,Percentage
1822,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2015.0,47.800000,Percentage
1823,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2016.0,46.615466,Percentage
1824,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2017.0,45.300000,Percentage


In [276]:
tot_df = pd.DataFrame(columns=['indicator', 'units', 'region', 'year', 'total_calc', 'total_rep'])
for ind in total_inds:
    try:
        df = socioecon[socioecon['indicator']==ind]
        for unit in df['units'].unique():
            dfu = df[df['units']==unit]
            for reg in dfu['Region'].unique():
                dfur = dfu[dfu['Region']==reg]
                for year in dfur['Year'].unique():
                    dfury = dfur[dfur['Year']==year]
                    dfury_g = dfury.groupby('category_1').sum()
                    tot_dict = {'indicator': ind,
                                'units': unit,
                                'region': reg,
                                'year': year,
                                'total_calc': dfury_g['value'].sum()-dfury_g.loc['Total', 'value'], #total calculated from the sum of all categories
                                'total_rep': dfury_g.loc['Total', 'value'] #reported total
                               }
                    tot_df = tot_df.append(tot_dict, ignore_index = True)
    except:
        pass

tot_df['diff'] = ((tot_df['total_rep']-tot_df['total_calc'])*100)/tot_df['total_rep'] #difference between reported and calculated totals

In [282]:
eq_tot = []
not_eq_tot = []

for ind in total_inds:
    if ind not in tot_df['indicator'].unique():
        print(ind, 'failed')
    if ind not in tot_df[(tot_df['diff']>5)|(tot_df['diff']<-5)]['indicator'].unique():
        print(ind, 'eq')
        eq_tot.append(ind)
    if ind in tot_df[(tot_df['diff']>5)|(tot_df['diff']<-5)]['indicator'].unique():
        print(ind, 'not_eq')
        not_eq_tot.append(ind)

Consumption of Chemical Fertilizers not_eq
Number of Reservoirs eq
Capacity of Reservoirs not_eq
Building Floorspace not_eq
Employment eq
Employed People in urban areas eq
International Trade of Goods not_eq
Exports by SITC not_eq
Imports by SITC not_eq
International Trade (by region) eq
Length of Railways in Operation not_eq
Length of Highways not_eq
Length of Regular Civil Aviation Routes not_eq
Passenger Traffic eq
Passenger Traffic per kilometer eq
Freight Traffic not_eq
Travel Distance per passenger not_eq
Average Freight Transport Distance not_eq
Civil Vehicles not_eq
Motor Drivers not_eq
Private Vehicles not_eq
Air Passenger Traffic not_eq
Air Freight Traffic not_eq
Total Air Traffic eq
Flying Time of General Aviation not_eq


In [1]:
for ind in not_eq_tot:
    print(ind, tot_df[tot_df['indicator']==ind]['diff'].max(), tot_df[tot_df['indicator']==ind]['diff'].min())

NameError: name 'not_eq_tot' is not defined

In [287]:
tot_df[tot_df['indicator']=='Air Passenger Traffic']

,indicator,units,region,year,total_calc,total_rep,diff
1214,Air Passenger Traffic,10 000 persons,China,1990.0,1.660000e+03,1.660000e+03,0.000000
1215,Air Passenger Traffic,10 000 persons,China,2000.0,7.125041e+03,6.721660e+03,-6.001207
1216,Air Passenger Traffic,10 000 persons,China,2010.0,2.744152e+04,2.676914e+04,-2.511750
1217,Air Passenger Traffic,10 000 persons,China,2017.0,5.618299e+04,5.515611e+04,-1.861763
1218,Air Passenger Traffic,10 000 persons,China,2018.0,6.230086e+04,6.117377e+04,-1.842436
1219,Air Passenger Traffic,10 000 persons,China,2019.0,6.710098e+04,6.599342e+04,-1.678288
1220,Air Passenger Traffic,10 000 persons/km,China,1990.0,2.304797e+06,2.304797e+06,0.000000
1221,Air Passenger Traffic,10 000 persons/km,China,2000.0,1.020784e+07,9.705437e+06,-5.176528
1222,Air Passenger Traffic,10 000 persons/km,China,2010.0,4.137178e+07,4.038996e+07,-2.430845
1223,Air Passenger Traffic,10 000 persons/km,China,2017.0,9.661284e+07,9.513036e+07,-1.558372


In [261]:
pwd

'/Users/oscaresbri/Documents/sci_team_data_bank/Projects/green-energy-data-platform/data/Socio-economic Data'

In [262]:
socioecon.to_csv('socioecon_cleanv2.csv', index=False)